In [1]:
#1、导包
import numpy as np
import re
import sys
import pandas as pd  #处理csv文件
#import whois        #获取域名注册信息
import tldextract  #Top Level Domain extract, 顶级域名提取
from urllib.parse import urlparse  #将url字符串拆分
import ipaddress as ip 
import dns.resolver
from matplotlib import colors
from sklearn import svm            
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
import matplotlib as mpl
import string

In [2]:
#2、数据采集

#加载csv文件里的url
def load_data(filepath):
    data=pd.read_csv(filepath,nrows=5000) #读取5000条数据
    urls=data[['url']]
    return urls
malicious_urls=load_data('phishing_verified_online.csv')            
benign_domains=load_data('top-1m.csv')
print(malicious_urls)
print(benign_domains)

                                                 url
0  http://beautifulattendee.com/css/css/aol$MV4/i...
1               http://www.grupbf18.whatsap-gr33.tk/
2                    http://bkpwhyaaa.hoaztbokep.ga/
3                http://www.bkpwhyaaa.hoaztbokep.ga/
4                    http://whattsap.claimfreedm.gq/
5                http://www.whattsap.claimfreedm.gq/
6                   http://grupbf18.whatsap-gr33.tk/
7  http://tickets-rech.000webhostapp.com/checkcar...
8  http://gdmusic.net/wps-config/customer_center/...
9  http://tickets-rech.000webhostapp.com/payrecha...
            url
0    google.com
1   youtube.com
2  facebook.com
3     tmall.com
4     baidu.com
5        qq.com
6      sohu.com
7    taobao.com
8        360.cn
9        jd.com


In [3]:
#提取域名
def extract_domain(urls):
    temp_domains=[]
    domains=[]
    for url in urls:
        parts=urlparse(url)
        #val=tldextract.extract(url) 效果不好
        temp_domain=parts.netloc
        if re.match('www.',temp_domain):
            domain=temp_domain.lstrip('www.')   #去除提取的初步域名的起始字符www.
            temp_domains.append(domain)
        else:
            temp_domains.append(temp_domain)
    for i in temp_domains:      #去重
        if i not in domains:
            domains.append(i)
    return domains
malicious_domains=extract_domain(malicious_urls['url'])
#print(malicious_domains)
malicious_domains=pd.DataFrame(malicious_domains)
benign_domains=pd.DataFrame(benign_domains)
malicious_domains.columns=['url']
#print(malicious_domains.head(10))
#print(benign_domains.head(10))

In [4]:
#添加标签
malicious_domains.insert(1,'malicious',np.ones(malicious_domains.shape[0]))
benign_domains.insert(1,'malicious',np.zeros(benign_domains.shape[0]))
malicious_domains.head(10)

,url,malicious
0,beautifulattendee.com,1.0
1,grupbf18.whatsap-gr33.tk,1.0
2,bkpwhyaaa.hoaztbokep.ga,1.0
3,whattsap.claimfreedm.gq,1.0
4,hattsap.claimfreedm.gq,1.0
5,tickets-rech.000webhostapp.com,1.0
6,gdmusic.net,1.0


In [5]:
#恶意非恶意域名合并
all_domains=pd.concat([malicious_domains,benign_domains])
all_domains.rename(index=str,columns={'url':'domain'},inplace = True)
all_domains.head(10)   #查看前10条数据

,domain,malicious
0,beautifulattendee.com,1.0
1,grupbf18.whatsap-gr33.tk,1.0
2,bkpwhyaaa.hoaztbokep.ga,1.0
3,whattsap.claimfreedm.gq,1.0
4,hattsap.claimfreedm.gq,1.0
5,tickets-rech.000webhostapp.com,1.0
6,gdmusic.net,1.0
0,google.com,0.0
1,youtube.com,0.0
2,facebook.com,0.0


In [ ]:
#3、特征提取(针对词法特征、域名网络特征)

#特征1：是否存在ip地址
'''def isip(domain):
    if ip.ip_address(domain):
        return 1
    else:
        return 0
all_domains['isip']=all_domains['domain'].apply(isip)
'''
#特征2：分隔符个数
def parse_count(domain):
    return domain.count('.')
all_domains['parse_count']=all_domains['domain'].apply(parse_count)

#特征3：连字符个数
def hyphen_count(domain):
    return domain.count('-')
all_domains['hyphen_count']=all_domains['domain'].apply(hyphen_count)

#特征4：数字占域名总长度的比例
def digit_length_percent(domain):
    percent=0.00
    num_regx=re.compile(r'[0-9]')
    numlist=num_regx.findall(domain)
    countNum=len(numlist)
    totalNum=len(domain)
    percent=countNum/totalNum
    return percent
all_domains['digit_length_percent']=all_domains['domain'].apply(digit_length_percent)

#特征5：域名总长度
def length_count(domain):
    return len(domain)
all_domains['length']=all_domains['domain'].apply(length_count)

#特征6：数字字母转换频率
def digit_letter_transfer(domain):
    pattern1=re.compile(r'[a-z][0-9]',re.I)
    pattern2=re.compile(r'[0-9][a-z]',re.I)
    result1=pattern1.findall(domain)
    result2=pattern2.findall(domain)
    return len(result1)+len(result2)
all_domains['digit_letter_transfer']=all_domains['domain'].apply(digit_letter_transfer)

#特征7：分隔符内数字个数的最大值
def max_one(nums):  #返回最大值
    max_num=0
    for num in nums:
        if num>max_num:
            max_num=num
    return max_num
def max_digit_length(domain):
    nums=[]
    max_num=0
    for i in domain:
        if i=='.':
            nums.append(max_num)
            max_num=0
        else:
            if i.isdigit():
                max_num+=1
            else:
                max_num+=0
    return max_one(nums) 
all_domains['max_digit_length']=all_domains['domain'].apply(max_digit_length)

#特征8：分隔符内字符的最大个数
def max_sub_length(domain):
    nums=[]
    max_num=0
    for i in domain:
        if i=='.':
            nums.append(max_num)
            max_num=0
        else:
            max_num+=1
    return max_one(nums) 
all_domains['max_sub_length']=all_domains['domain'].apply(max_sub_length)
#特征9：域名请求类型
def request_type(domain,tcpfallback=True,timeout=30):
    sum=0
    qtype=['A','MX','CNAME','SOA','NS','PTR']  #只选择几种常用的RR资源类型
    for type in qtype:
        try:
            answers=dns.resolver.resolve(domain,type)
            sum+=1
        except Exception as e:
            sum+=0
    return sum/6
all_domains['RR_type']=all_domains['domain'].apply(request_type)
all_domains.tail(10)

In [7]:
#4、数据分割
z,y,x=np.split(all_domains,
             (1,2,),
             axis=1)
#print(x)
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,              #所要划分的样本特征集
                                                               y,              #所要划分的样本结果
                                                               random_state=1, #随机数种子
                                                               test_size=0.3)  #测试样本占比
#print(y_test)

In [8]:
#5、模型搭建
#SVM分类器构建
def classifier():
    #clf = svm.SVC(C=0.8,kernel='rbf', gamma=50,decision_function_shape='ovr')
    clf = svm.SVC(C=0.5,                         #误差项惩罚系数,默认值是1
                  kernel='linear',               #线性核 kenrel="rbf":高斯核
                  decision_function_shape='ovr') #决策函数
    return clf
#SVM模型定义
clf = classifier() 
#训练模型   
def train(clf,x_train,y_train):
    clf.fit(x_train,         #训练集特征向量
            y_train.values.astype('int').ravel()) #训练集目标值
train(clf,x_train,y_train)

In [9]:
#6、模型评估
    
def print_accuracy(clf,x_test,y_test):
    #精确率
    accscore = clf.score(x_test, y_test.values.astype('int').ravel())
    print("精确率："+str(accscore))
def print_recall(clf,x_test,y_test):
    y_true = y_test  # 实际上的标签
    y_predict = clf.predict(x_test)  # 模型根据特征预测的标签
    #print(y_true.shape)
    #print(y_predict.shape)
    print("召回率："+str(recall_score(y_true.values.astype('int').ravel(), y_predict, average='macro')))
    
print_accuracy(clf,x_test,y_test)
print_recall(clf,x_test,y_test)

精确率：0.9030927835051547
召回率：0.8858134632160513


In [ ]:
#7、模型运用
print('Please input the correct domain format. For example,baidu.com instead of www.baidu.com')
test_domain= input("input:")
test = [[parse_count(test_domain),hyphen_count(test_domain),digit_length_percent(test_domain),length_count(test_domain),digit_letter_transfer(test_domain),max_digit_length(test_domain),max_sub_length(test_domain),request_type(test_domain)]]
test_data= pd.DataFrame(test, columns=['parse_count','hyphen_count','digit_length_percent','length','digit_letter_transfer','max_digit_length','max_sub_length','RR_type'])
print(test_data)

Please input the correct domain format. For example,baidu.com instead of www.baidu.com


In [11]:
#预测
predict = clf.predict(test_data)
if(predict == 1):
    print("malicious domain!")
else:
    print("benign domain!")

benign domain!
